In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)


In [6]:
scales = [0]
selected_columns = ['x1', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    y = data[1]
    # FIXME: should be replaced by `s.where` when it's available. Now the result may be incorrect
    filter = (y < 20)
    # FIXME: not sure how to do filtering correctly here
    filtered_x = torch.where(filter, x, 0.0)
    filtered_y = torch.where(filter, y, 0.0)
    return s.linear_regression(filtered_x, filtered_y)


error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-15208,13532],"logrows":15,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":7872,"total_assignments":426,"total_const_size":5,"model_instance_shapes":[[1],[1,2,1]],"model_output_scales":[0,0],"model_input_scales":[0,0],"module_sizes":{"kzg":[],"poseidon":[7872,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 6.506374835968018 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 2.0
witness result 2 : 10.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [3898461358030585804, 15381148731429954174, 10354293334678834451, 3143530863341041542]]], 'proof': '2b7d33349eed5e95d7471c71f1dd3e70b08372206ae05ac37daa9b8347ae2aa905d5fd6ca3b35a98e66f93824964e83486a967a33bc5434ef3b12c7ab548343516c9d1ef04e281285042a86fa49a3a6da741a5cde6a27939a970eb6ef54ef63127d5ccc6c64c985c106c2d89d078ca7422aa667ad199c8f7f428bbc86828a9941f24f52d5124c5f606d672983f95a2f37c8b270ebcf2ff0d114fb3670698e75702eea6bf9720ebfdfab90cb28b44498daa062b9d1ba14bff44e9323b70c132f7051eb48098deb68a3c7a485954f5211df4c47e8f43e9029e135c713f005aa4d20269f73271b58fd02a667cca44671113ba7d2c1613354541c0acb57ed15cd40516974ce4ea5b721e44

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 2.0
